In [7]:
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
def plot_loss(epoch, model, crit, optim, s, w, y, **kwargs):
    df = pd.read_csv(f'models-generation-{epoch}epochs/{model}-{crit}-{optim}-s{s}-w{w}/{model}-{crit}-{optim}-s{s}-w{w}.csv')
    df.plot(y=y, xticks=range(0, epoch + 1, 2), ylabel="Loss", xlabel="Epoch", **kwargs)

In [9]:
for y in [('train_loss', 'training'), ('test_loss', 'testing')]:
    for crit in ["BCE", "MSE"]:
        fig, ax = plt.subplots()
        
        for w in [100, 200, 400, 800, 'None']:
            plot_loss(epoch=25, model="RNN-relu", crit=crit, optim="ADAM", s=100, w=w, y=y[0], ax=ax)

        ax.legend(['Window size 100', 'Window size 200', 'Window size 400', 'Window size 800', 'Window size none'])
        ax.set_title(f"Impact of the window on the loss ({y[1]},{crit})\n RNN-relu, Adam, step size 100")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        plt.savefig(f"window-impact-{crit.lower()}-{y[1]}.png")
        plt.close(fig)

In [10]:
for y in [('train_loss', 'training'), ('test_loss', 'testing')]:
    for crit in ["BCE", "MSE"]:
        fig, ax = plt.subplots()

        for s in [100,200,400,800]:
            plot_loss(epoch=25, model="RNN-relu", crit=crit, optim="ADAM", s=s, w=800, y=y[0], ax=ax)
            
        plt.legend(['Step size 100', 'Step size 200', 'Step size 400', 'Step size 800'])
        plt.title(f"Impact of the step on the loss ({y[1]},{crit})\n RNN-relu, Adam, window size 800")
        plt.savefig(f"step-impact-{crit.lower()}-{y[1]}.png")
        plt.close(fig)

In [11]:
def get_mean(epoch, model, crit, optim, s, w, y):
    df = pd.read_csv(f'models-generation-{epoch}epochs/{model}-{crit}-{optim}-s{s}-w{w}/{model}-{crit}-{optim}-s{s}-w{w}.csv')
    return df.mean()[y]

print("Average epoch time per step size")
print("RNN-tanh-BCE-ADAM-sx-w8")
for s in [100,200,400,800]:
    m = get_mean(epoch=25, model="RNN-tanh", crit="BCE", optim="ADAM", s=s, w=800, y="epoch_time")
    print(f'Step size of {s} : {m}')

print("Average epoch time per window size")
print("RNN-tanh-BCE-ADAM-s1-wx")
for w in [100,200,400,800,'None']:
    m = get_mean(epoch=25, model="RNN-tanh", crit="BCE", optim="ADAM", s=100, w=w, y="epoch_time")
    print(f'Window size of {w} : {m}')

Average epoch time per step size
RNN-tanh-BCE-ADAM-sx-w8
Step size of 100 : 92.9175597858429
Step size of 200 : 47.14063490867615
Step size of 400 : 24.181218681335448
Step size of 800 : 12.896559791564941
Average epoch time per window size
RNN-tanh-BCE-ADAM-s1-wx
Window size of 100 : 20.734648303985594
Window size of 200 : 31.5911381816864
Window size of 400 : 53.481856956481934
Window size of 800 : 92.9175597858429
Window size of None : 418.1322982120514


In [12]:
for crit in ["BCE", "MSE"]:
    fig, ax = plt.subplots()
    # Add padding using subplots_adjust
    if crit == "BCE":
        padding = 0.115
    else:
        padding = 0.125
    plt.subplots_adjust(left=padding, right=1-padding)

    for m in ['RNN-relu', 'RNN-tanh', 'LSTM', 'GRU']:
        plot_loss(epoch=50, model=m, crit=crit, optim="ADAM", s=100, w=800, y='train_loss', ax=ax, linestyle="solid")

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Train loss")
    
    lines, labels = ax.get_legend_handles_labels()
    custom_lines = [plt.Line2D([0], [0], color='black', linestyle='dotted', markersize=10, label='Test'),
                    plt.Line2D([0], [0], color='black', linestyle='solid', markersize=10, label='Train')]
    
    ax.legend(lines + custom_lines, ['RNN-relu', 'RNN-tanh', 'LSTM', 'GRU'] + ['Test', 'Train'])
    ax.set_title(f"Impact of the model on the loss ({crit})\n Adam, step size 100, window size 800")

    ax = ax.twinx()
    for m in ['RNN-relu', 'RNN-tanh', 'LSTM', 'GRU']:
        plot_loss(epoch=50, model=m, crit=crit, optim="ADAM", s=100, w=800, y='test_loss', ax=ax, linestyle="dotted")
    
    ax.set_ylabel("Test loss")
    ax.get_legend().remove()
    plt.savefig(f"model-impact-{crit.lower()}.png")
    plt.close(fig)